In [1]:
# Donchian Weekly Classic Trend Following System
import gta_price
import numpy as np
import pandas as pd

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'PHTM'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
p = [48, 24, 12, 6] # Look back periods.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2 # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.0  # Stamp Duty percentage.

In [3]:
# Function definitions.
def charges(date, commission):
    '''Calculate trading charges.'''
    df = pd.DataFrame(index=date, columns=['date_shift', 'charges'])
    df.date_shift = df.index.values
    df.date_shift = df.date_shift.shift()
    df.charges = np.where(df.index == df.date_shift, 0, commission)
    return df.charges

def donchian(prices, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame()
    df['upr'] = prices.high.rolling(period).max().shift(periods=1)
    df['lwr'] = prices.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    return df

def shares_div4(shares):
    '''Modify number of shares to be purchased to be divisible by 4.'''
    s = pd.Series(shares.values)
    for i in s.index:
        while s.iloc[i] % 4 != 0:
            s.iloc[i] += 1
    return shares

def state(entry_signal, exit_signal, period):
    '''Calculate trade state signals.'''
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ['entry', 'exit']
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'entry'] == 1 \
        and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'exit'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    return df.state

def trade_summary(trade_list):
    '''Generate trade summary.'''
    frame = pd.DataFrame(columns=['trade', 'entry', 'volatility','cost', 'exit', 'days', 'profit', 'pct', 'annual'])
    if trade_list.entry.iloc[-1] == 1:
        trade_num = trade_list.index.max()
    else:
        trade_num = trade_list.index.max() + 1
        
    for i in list(range(1, trade_num)):
        df = trade_list.loc[i]
        trade = df.index[0]
        entry = df.date.iloc[0]
        volatility = df.volatility.iloc[0]
        cost = df.cost.iloc[0]
        exit = df.date.iloc[-1]
        days = df.days.iloc[-1]
        profit = df.profit.sum()
        pct = (profit / cost) * 100
        annual = ((1 + pct / 100) ** (365 / days) - 1) * 100
        frame.loc[i] = [trade, entry, volatility, cost, exit, days, profit, pct, annual]
    frame = frame.set_index('trade')
    return frame

pd.set_option("display.max_columns", None)

In [4]:
# Import weekly closing prices.
prices = gta_price.weekly(exchange, tidm)

In [5]:
# Donchian channels.
for i in range(len(p)):
    globals()[f'p{i + 1}'] = p[i]
    globals()[f'dc{i + 1}'] = donchian(prices, p[i])

In [6]:
# System 1 entries & exits.
s1 = pd.concat([prices, dc1], axis=1)
s1['sys'] = 1
s1['buy'] = np.where(s1.close > s1.upr, 1, 0)
s1['sell'] = np.where(s1.close < s1.mid, 1, 0)
s1['state'] = state(s1.buy, s1.sell, p1)
s1['entry'] = np.where(np.logical_and(s1.state == 1, s1.state.shift(periods=1) == 0), 1, 0)
s1['exit'] = np.where(np.logical_and(s1.state == 0, s1.state.shift(periods=1) == 1), 1, 0)

In [7]:
# System 2 entries & exits.
s2 = pd.concat([prices, dc2], axis=1)
s2['sys'] = 2
s2['buy'] = s1.entry
s2['sell'] = np.where(s2.close < s2.mid, 1, 0)
s2['state'] = state(s2.buy, s2.sell, p2)
s2['entry'] = np.where(np.logical_and(s2.state == 1, s2.state.shift(periods=1) == 0), 1, 0)
s2['exit'] = np.where(np.logical_and(s2.state == 0, s2.state.shift(periods=1) == 1), 1, 0)

In [8]:
# System 3 entries & exits.
s3 = pd.concat([prices, dc3], axis=1)
s3['sys'] = 3
s3['buy'] = s1.entry
s3['sell'] = np.where(s3.close < s3.mid, 1, 0)
s3['state'] = state(s3.buy, s3.sell, p3)
s3['entry'] = np.where(np.logical_and(s3.state == 1, s3.state.shift(periods=1) == 0), 1, 0)
s3['exit'] = np.where(np.logical_and(s3.state == 0, s3.state.shift(periods=1) == 1), 1, 0)

In [9]:
# System 4 entries & exits.
s4 = pd.concat([prices, dc4], axis=1)
s4['sys'] = 4
s4['buy'] = s1.entry
s4['sell'] = np.where(s4.close < s4.mid, 1, 0)
s4['state'] = state(s4.buy, s4.sell, p4)
s4['entry'] = np.where(np.logical_and(s4.state == 1, s4.state.shift(periods=1) == 0), 1, 0)
s4['exit'] = np.where(np.logical_and(s4.state == 0, s4.state.shift(periods=1) == 1), 1, 0)

In [10]:
# Trade list indexed by date.
td = pd.concat([s1[s1.entry == 1] , s1[s1.exit == 1], s2[s2.exit == 1], s3[s3.exit == 1], s4[s4.exit == 1]], axis=0)
td = td.sort_index()

In [11]:
# Position size (buy).
td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close), 0)
td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility), 0)
td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)
td.shares = shares_div4(td.shares) # Modify number of shares to be purchased to be divisible by 4.
td.risk_amt = np.where(td.entry == 1, (td.close * td.shares), 0) # Adjust risk amount for revised share count.

In [12]:
# Position size (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        shares = row['shares']
    elif row['exit'] == 1:
        td.at[index, 'shares'] = int(shares / 4)

In [13]:
# Charges.
td['charges'] = charges(td.index, commission)

In [14]:
# Stamp duty.
td['sduty'] = np.where(td.entry==1, ((sduty / 100) * td.close * td.shares), 0)

In [15]:
# Cost (buy).
td['cost'] = np.where(td.entry == 1, ((td.close * td.shares) + td.charges + td.sduty), 0)

In [16]:
# Cost (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        cost = row['cost']
    elif row['exit'] == 1:
        td.at[index, 'cost'] = (cost / 4)

In [17]:
# Value (sell).
td['value'] = np.where(td.exit == 1, ((td.close * td.shares) - td.charges), 0)

In [18]:
# Profit.
td['profit'] = np.where(td.exit == 1, td.value - td.cost, 0)

In [19]:
# Cumulative profit.
td['profit_cum'] = td.profit.cumsum()

In [20]:
# Percentage return.
td['pct'] = np.where(td.exit == 1, ((td.profit / td.cost) * 100), 0)

In [21]:
# Trade duration.
td['days'] = 0
for index, row in td.iterrows():
    if row['entry'] == 1:
        start_date = index
    elif row['exit'] == 1:
        td.at[index, 'days'] = index - start_date
td.days = td.days.astype('timedelta64[D]')
td.days = td.days.dt.days

In [22]:
# Annual percentage return.
td['annual'] = ((np.power(1 + td.profit / td.cost, (365 / td.days)) - 1) * 100)

In [23]:
# Trade list indexed by trade.
td['trade'] = td.state.cumsum()
td = td.reset_index()
td = td.set_index('trade')
td

,date,open,high,low,close,upr,lwr,mid,sys,buy,sell,state,entry,exit,volatility,risk_amt,shares,charges,sduty,cost,value,profit,profit_cum,pct,days,annual
trade,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2003-03-21,0.2450,0.2800,0.2425,0.2775,0.2650,0.1350,0.20000,1,1,0,1,1,0,0.279279,5371.290,19356,11.95,0.0,5383.2400,0.0000,0.0000,0.0000,0.000000,0,0.000000
1,2003-09-12,1.1200,1.1325,1.0300,1.0350,1.2200,1.0100,1.11500,4,0,1,0,0,1,0.000000,0.000,4839,11.95,0.0,1345.8100,4996.4150,3650.6050,3650.6050,271.257087,175,1442.335197
1,2003-11-21,1.0800,1.0900,0.9250,1.0000,1.2200,0.9225,1.07125,3,0,1,0,0,1,0.000000,0.000,4839,11.95,0.0,1345.8100,4827.0500,3481.2400,7131.8450,258.672472,245,570.481633
1,2004-04-30,1.2700,1.3100,1.1900,1.1900,1.4575,0.9250,1.19125,2,0,1,0,0,1,0.000000,0.000,4839,11.95,0.0,1345.8100,5746.4600,4400.6500,11532.4950,326.988951,406,268.768548
1,2004-05-21,0.9825,0.9825,0.8800,0.9450,1.4575,0.4900,0.97375,1,0,1,0,0,1,0.000000,0.000,4839,11.95,0.0,1345.8100,4560.9050,3215.0950,14747.5900,238.896650,427,183.858565
2,2005-07-22,1.1000,1.1400,1.1000,1.1375,1.1300,0.8350,0.98250,1,1,0,1,1,0,0.136264,11011.000,9680,11.95,0.0,11022.9500,0.0000,0.0000,14747.5900,0.000000,0,0.000000
2,2005-09-23,1.2500,1.2500,1.2100,1.2250,1.2800,1.1800,1.23000,4,0,1,0,0,1,0.000000,0.000,2420,11.95,0.0,2755.7375,2952.5500,196.8125,14944.4025,7.141918,63,49.133217
2,2005-10-07,1.1650,1.1825,1.1475,1.1500,1.2800,1.1000,1.19000,3,0,1,0,0,1,0.000000,0.000,2420,11.95,0.0,2755.7375,2771.0500,15.3125,14959.7150,0.555659,77,2.661478
2,2006-01-06,1.2250,1.2450,1.2000,1.2200,1.3600,1.1000,1.23000,2,0,1,0,0,1,0.000000,0.000,2420,11.95,0.0,2755.7375,2940.4500,184.7125,15144.4275,6.702834,168,15.137186


In [24]:
# Trade summary.
tds = trade_summary(td)
tds.round({'volatility': 3, 'pct': 1, 'annual': 1})

,entry,volatility,cost,exit,days,profit,pct,annual
trade,,,,,,,,
1,2003-03-21,0.279,5383.24,2004-05-21,427,14747.5900,274.0,208.8
2,2005-07-22,0.136,11022.95,2006-01-13,175,-319.9000,-2.9,-6.0
3,2009-04-03,0.363,4150.35,2009-05-15,42,-1148.0450,-27.7,-94.0
4,2009-08-14,0.403,3735.40,2010-06-11,301,1036.7000,27.8,34.6
5,2010-08-20,0.330,4555.11,2011-02-25,189,508.1450,11.2,22.7
6,2013-01-25,0.253,5946.97,2014-09-26,609,3011.0250,50.6,27.8
7,2015-10-09,0.117,12808.23,2016-02-12,126,-819.7775,-6.4,-17.4
8,2016-02-26,0.136,11040.31,2016-05-06,70,-698.1700,-6.3,-28.9
9,2017-06-16,0.118,12710.41,2018-02-09,238,-900.9560,-7.1,-10.7


In [25]:
cost_total = tds.cost.sum()
cost_total

79283.48

In [26]:
profit_total = tds.profit.sum()
profit_total

15633.881499999996

In [27]:
pct_total = (profit_total / cost_total) * 100
pct_total

19.71896478307965

In [28]:
days_total = (tds.exit.iloc[-1] - tds.entry.iloc[0])
days_total = days_total.days
days_total

6769

In [29]:
annual_total = ((1 + pct_total / 100) ** (365 / days_total) - 1) * 100
annual_total

0.9752008335343509